# Q-Learning Frozen Lake

In [26]:
#pip install --upgrade setuptools

In [27]:
#!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit2/requirements-unit2.txt

In [2]:
%%capture
!sudo apt-get update
!apt install python-opengl ffmpeg xvfb
!pip3 install pyvirtualdisplay

In [ ]:
import os
os.kill(os.getpid(), 9)

# Importar Modulos

In [3]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

#import pickle5 as pickle
from tqdm.notebook import tqdm

In [4]:
# Create the FrozenLake-v1 environment using 4x4 map and non-slippery version and render_mode="rgb_array"
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")

In [5]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Discrete(16)
Sample observation 10


In [6]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


In [33]:
type(env.action_space.sample())

numpy.int64

In [7]:
state_space = env.observation_space.n
print("Hay", state_space, "posibles estados")

action_space = env.action_space.n
print("Hay", action_space, "posibles acciones")

Hay 16 posibles estados
Hay 4 posibles acciones


## Creacion Q-table

In [8]:
# Let's create our Qtable of size (state_space, action_space) and initialized each values at 0 using np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [9]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)
Qtable_frozenlake

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

## Politica greedy

In [10]:
def greedy_policy(Qtable, state):
  # Exploitation: take the action with the highest state, action value
  action = np.argmax(Qtable[state][:]) # indice del maximo valor en la fila state de la matriz 
  
  return action

In [12]:
# Ejemplo de la funcion gree_policy
greedy_policy([[0,1,4,0] , [0,0,0,0]], 0)

2

## Politica epsilon-greedy

In [14]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Randomly generate a number between 0 and 1
  random_int = random.uniform(0,1)
  # if random_int > greater than epsilon --> exploitation
  if random_int > epsilon:
    # Take the action with the highest value given a state
    # np.argmax can be useful here
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()
  
  return action

## Hiperparametros

In [15]:
# Training parameters
n_training_episodes = 1000  # Total training episodes
learning_rate = 0.7          # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate
eval_seed = []               # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0            # Exploration probability at start
min_epsilon = 0.05           # Minimum exploration probability 
decay_rate = 0.005          # Exponential decay rate for exploration prob

In [16]:
env.reset()

(0, {'prob': 1})

## Bucle de Entrenamiento

In [17]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, Qtable): #max_steps
  for episode in tqdm(range(n_training_episodes)):   # tqdm: muestra barra de progreso
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    
    # Reset the environment
    state, info = env.reset()
    step = 0
    terminated = False
    truncated = False
    
    while not (terminated or truncated):
    #for step in range(max_steps):
      # Choose the action At using epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, terminated, truncated, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr*[R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      # Update Q(s,a):= (1-lr)*Q(s,a) + lr*[R(s,a) + gamma * max Q(s',a')]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])    
    
      # If terminated or truncated finish the episode
      #if terminated or truncated:
      #  break
      
      # Our next state is the new state
      state = new_state
  return Qtable

In [20]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, Qtable_frozenlake)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [21]:
# Tabla Q despues del entrenamineto
# Nuestra politica que defeinica como una plotica avariciosa
Qtable_frozenlake

array([[0.73509189, 0.77378094, 0.6983373 , 0.73509189],
       [0.73509189, 0.        , 0.66010819, 0.69772665],
       [0.69769945, 0.56409742, 0.20052806, 0.20063146],
       [0.24864612, 0.        , 0.04692491, 0.        ],
       [0.77378094, 0.81450625, 0.        , 0.73509189],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.89951996, 0.        , 0.14376491],
       [0.        , 0.        , 0.        , 0.        ],
       [0.81450625, 0.        , 0.857375  , 0.77378094],
       [0.81450625, 0.9025    , 0.9025    , 0.        ],
       [0.85618157, 0.95      , 0.        , 0.83613866],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.95      , 0.857375  ],
       [0.9025    , 0.95      , 1.        , 0.9025    ],
       [0.        , 0.        , 0.        , 0.        ]])

## Evaluacion del agente

In [22]:
def evaluate_agent(env, n_eval_episodes, Q, seed): #max_steps
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param Q: The Q-table
  :param seed: The evaluation seed array (for taxi-v3)
  """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    #step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0
    
    while not (terminated or truncated):
    #for step in range(max_steps):
      # Take the action (index) that have the maximum expected future reward given that state
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward
        
      #if terminated or truncated:
      #  break
      state = new_state
    
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [23]:
mean_reward, std_reward = evaluate_agent(env, 1000, Qtable_frozenlake, eval_seed) #max_steps
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Mean_reward=1.00 +/- 0.00


## Grabar Episodio

In [24]:
def record_video(env, Qtable, out_directory, fps=1):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  images = []  
  terminated = False
  truncated = False
  state, info = env.reset(seed=random.randint(0,500))
  img = env.render()
  images.append(img)
  while not terminated or truncated:
    # Take the action (index) that have the maximum expected future reward given that state
    action = np.argmax(Qtable[state][:])
    state, reward, terminated, truncated, info = env.step(action) # We directly put next_state = state for recording logic
    img = env.render()
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [25]:
record_video(env, Qtable_frozenlake, 'prueba10.gif')